<a href="https://colab.research.google.com/github/RozminWaghu99/Recommendation_system/blob/main/Hotel_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Recommendation System 

A hotel recommendation system aims to predict which hotel a user is most likely to choose from among all hotels. So to build this type of system which will help the user to book the best hotel out of all the other hotels. We can do this using customer reviews.

As we are going to build a recommendation system according to the user ratings so here I will be using Natural Language Processing. Now let’s import the necessary Python libraries and the dataset to get started with the task of creating a hotel recommendation system:

In [59]:
import pandas as pd
import numpy as np
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 
from nltk.stem.wordnet import WordNetLemmatizer
import random

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading the hotel data

In [60]:
#reading hotel room attributes data
hra = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Project/Hotel Recommendation System/hotel data/Hotel_Room_attributes.csv")
hra.head()

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"


In [61]:
#reading hotels details
hd = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Project/Hotel Recommendation System/hotel data/Hotel_details.csv")
hd.head()

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [62]:
#reading hotel room price details
hrp = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Project/Hotel Recommendation System/hotel data/hotels_RoomPrice.csv")
pd.set_option("display.max_columns", len(hrp.columns))
hrp.head()

,id,refid,hotelcode,websitecode,dtcollected,ratedate,los,guests,roomtype,onsiterate,netrate,currency,ratedescription,ratetype,sourceurl,roomamenities,maxoccupancy,ispromo,closed,discount,promoname,status_code,taxstatus,taxtype,taxamount,proxyused,israteperstay,mealinclusiontype,hotelblock,input_dtcollected
0,50677497,8646773,634876,5,2019-10-12 15:46:54,2019-11-02,1,1,Double Room,82.36,87.84,USD,"Room size: 15 m²/161 ft², Shower, 1 king bed",Cancellation policy,https://www.agoda.com/en-gb/the-old-cider-hous...,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,1,Y,N,6.24,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.165.126:80,Y,Free Breakfast,NaN,2019-10-13
1,50672149,6234691,8328096,5,2019-10-12 15:47:45,2019-11-30,1,1,Vacation Home,636.09,0.00,USD,"Shower, Kitchenette, 2 bedrooms, 1 double bed ...",Cancellation policy,https://www.agoda.com/en-gb/pantlleni/hotel/be...,Air conditioning: ;Closet: ;Fireplace: ;Free W...,4,N,N,0.00,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.164.126:80,Y,NaN,NaN,2019-10-13
2,50643430,7015677,8323442,5,2019-10-12 15:47:00,2019-12-20,1,1,Vacation Home,591.74,0.00,USD,"Shower, Kitchenette, 2 bedrooms, 1 double bed ...",Extra low price! (non-refundable),https://www.agoda.com/en-gb/willow-cottage/hot...,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,4,N,N,0.00,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.166.122:80,Y,NaN,NaN,2019-10-13
3,50650317,7327094,7975,5,2019-10-12 15:47:02,2019-12-28,1,1,Standard Triple Room,881.48,0.00,USD,"Room size: 20 m²/215 ft², Shower, 3 single beds",Extra low price! (non-refundable),https://www.agoda.com/en-gb/apollo-hotel-londo...,Air conditioning: ;Clothes rack: ;Coffee/tea m...,1,N,N,0.00,NaN,200,2,Tax 20%,0.0,media:M3diAproxy@209.242.219.141:80,Y,NaN,NaN,2019-10-13
4,50650318,7327094,7975,5,2019-10-12 15:47:02,2019-12-28,1,1,Standard Triple Room,897.53,0.00,USD,"Room size: 20 m²/215 ft², Shower, 3 single beds",Extra low price! (non-refundable),https://www.agoda.com/en-gb/apollo-hotel-londo...,Air conditioning: ;Clothes rack: ;Coffee/tea m...,3,N,N,0.00,NaN,200,2,VAT $ 179.51,0.0,media:M3diAproxy@209.242.219.141:80,Y,Free breakfast for {3},NaN,2019-10-13


# Inspecting data file

In [63]:
print('Hotel room arttibutes data shape',hra.shape)
print('Hotel details data shape',hd.shape)
print('Hotel room price data shape',hrp.shape)

Hotel room arttibutes data shape (165873, 5)
Hotel details data shape (108048, 14)
Hotel room price data shape (165873, 30)


In [64]:
#check uniques values
for column in [hra,hd,hrp]:
    print(column.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165873 entries, 0 to 165872
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               165873 non-null  int64 
 1   hotelcode        165873 non-null  int64 
 2   roomamenities    161054 non-null  object
 3   roomtype         165873 non-null  object
 4   ratedescription  161054 non-null  object
dtypes: int64(2), object(3)
memory usage: 6.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108048 entries, 0 to 108047
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            108048 non-null  int64  
 1   hotelid       108048 non-null  int64  
 2   hotelname     108048 non-null  object 
 3   address       102955 non-null  object 
 4   city          108048 non-null  object 
 5   country       108048 non-null  object 
 6   zipcode       83486 non-null   float64
 7   p

# Check sum of any Duplicates in the data?

In [65]:
#sum of duplicates of hotle room attributes
print("sum of duplicates of hotle room attributes is ",hra.duplicated().sum())

sum of duplicates of hotle room attributes is  0


In [66]:
#sum of duplicates of hotle room price
print("sum of duplicates of hotle room price is ",hrp.duplicated().sum())

sum of duplicates of hotle room price is  0


In [67]:
#sum of duplicates of hotle room details
print("sum of duplicates of hotle room details is ",hd.duplicated().sum())

sum of duplicates of hotle room details is  0


# Data Formatting

In [68]:
#chnage hotel details column name to merge
hd.rename(columns = {'hotelid':'hotelcode'}, inplace = True)

#merge hotel details data and room arttibutes data based on hotel code
hotel_df = pd.merge(hd, hra, on='hotelcode')[['hotelcode','hotelname','roomtype','roomamenities','city','country','address','url','starrating']]
hotel_df.head()

,hotelcode,hotelname,roomtype,roomamenities,city,country,address,url,starrating
0,177234,Elounda Ilion Hotel,Double Room with Sea View,Air conditioning: ;Coffee/tea maker: ;Desk: ;F...,Crete Island,Greece,Main Street,https://www.agoda.com/en-gb/elounda-ilion-hote...,4
1,177234,Elounda Ilion Hotel,Double Room with Garden View,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Crete Island,Greece,Main Street,https://www.agoda.com/en-gb/elounda-ilion-hote...,4
2,177234,Elounda Ilion Hotel,Double Room with Garden View,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Crete Island,Greece,Main Street,https://www.agoda.com/en-gb/elounda-ilion-hote...,4
3,177234,Elounda Ilion Hotel,Double Room with Garden View,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Crete Island,Greece,Main Street,https://www.agoda.com/en-gb/elounda-ilion-hote...,4
4,177234,Elounda Ilion Hotel,Double Room with Garden View,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Crete Island,Greece,Main Street,https://www.agoda.com/en-gb/elounda-ilion-hote...,4


# Check any null data present in new data

In [69]:
#sum of null data in every column
print("sum of null data in every column\n",hotel_df.isna().sum())

sum of null data in every column
 hotelcode           0
hotelname           0
roomtype            0
roomamenities    5309
city                0
country             0
address           577
url                 0
starrating          0
dtype: int64


#Data cleaning

In [70]:
#onsite rate is one important feature which could be useful to recommend
#we will drop the rest since it is present in other table and we are going to merge the 
hrp.drop(['id','refid','websitecode','dtcollected','ratedate','los','guests','roomtype','netrate','ratedescription','ratetype','sourceurl','roomamenities'
,'closed','promoname','status_code','taxstatus','taxtype','taxamount','proxyused','israteperstay','hotelblock','input_dtcollected'],axis=1,inplace=True)
print(hrp.columns)

Index(['hotelcode', 'onsiterate', 'currency', 'maxoccupancy', 'ispromo',
       'discount', 'mealinclusiontype'],
      dtype='object')


In [71]:
#dropping null values from the data
hotel_df=hotel_df.dropna()
hotel_df.isna().sum()

hotelcode        0
hotelname        0
roomtype         0
roomamenities    0
city             0
country          0
address          0
url              0
starrating       0
dtype: int64

In [72]:
#grouping data to calculate avg mean of star rating
hotel_df = hotel_df.groupby(['hotelcode','hotelname','roomtype','city','address','roomamenities','country','url']).mean()['starrating'].reset_index()
hotel_df.head()

,hotelcode,hotelname,roomtype,city,address,roomamenities,country,url,starrating
0,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
1,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Blackout curtains: ;Coffee/...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
2,31,"The Harmonie Vienna, BW Premier Collection",Comfort Double Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
3,31,"The Harmonie Vienna, BW Premier Collection",Comfort Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Blackout curtains: ;Coffee/...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
4,31,"The Harmonie Vienna, BW Premier Collection",Double Non Smoking,Vienna,Harmoniegasse 5-7,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0


# POPULARITY BASED RECOMMENDATION SYSTEM

In [73]:
#popular hotel based on highest star rating
popular_df = hotel_df[hotel_df['starrating']>=3].sort_values('starrating',ascending=False).head(20)
popular_df

,hotelcode,hotelname,roomtype,city,address,roomamenities,country,url,starrating
0,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
24638,230619,Doubletree By Hilton Bristol South Cadbury House,Panoramic Suite,Bristol,"Cadbury House, Frost Hill, Congresb",Additional bathroom: ;Additional toilet: ;Air ...,United Kingdom,https://www.agoda.com/en-gb/doubletree-by-hilt...,4.0
24636,230619,Doubletree By Hilton Bristol South Cadbury House,King Family Guestroom,Bristol,"Cadbury House, Frost Hill, Congresb",Air conditioning: ;Alarm clock: ;Blackout curt...,United Kingdom,https://www.agoda.com/en-gb/doubletree-by-hilt...,4.0
24635,230619,Doubletree By Hilton Bristol South Cadbury House,King Deluxe Room,Bristol,"Cadbury House, Frost Hill, Congresb",Air conditioning: ;Alarm clock: ;Bathrobes: ;B...,United Kingdom,https://www.agoda.com/en-gb/doubletree-by-hilt...,4.0
24634,230619,Doubletree By Hilton Bristol South Cadbury House,King Deluxe Family Room,Bristol,"Cadbury House, Frost Hill, Congresb",Air conditioning: ;Alarm clock: ;Bathrobes: ;B...,United Kingdom,https://www.agoda.com/en-gb/doubletree-by-hilt...,4.0
24633,230619,Doubletree By Hilton Bristol South Cadbury House,King Bed Guestroom,Bristol,"Cadbury House, Frost Hill, Congresb",Air conditioning: ;Alarm clock: ;Blackout curt...,United Kingdom,https://www.agoda.com/en-gb/doubletree-by-hilt...,4.0
24627,229895,Hotel Riezler Hof,Standard Single Room,Riezlern,Walserstrasse 57,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/hotel-riezler-hof/...,4.0
24604,229781,Hotel Pelirocco,The Nookii Room Double Room,Brighton and Hove,10 Regency Square,Air conditioning: ;Bathrobes: ;Carpeting: ;Clo...,United Kingdom,https://www.agoda.com/en-gb/hotel-pelirocco/ho...,4.0
24603,229781,Hotel Pelirocco,The Bowery Room Standard Double Room,Brighton and Hove,10 Regency Square,Air conditioning: ;Bathrobes: ;Carpeting: ;Clo...,United Kingdom,https://www.agoda.com/en-gb/hotel-pelirocco/ho...,4.0
24602,229781,Hotel Pelirocco,Superior Double Room - Bettie's Boudoir,Brighton and Hove,10 Regency Square,Air conditioning: ;Bathrobes: ;Carpeting: ;Clo...,United Kingdom,https://www.agoda.com/en-gb/hotel-pelirocco/ho...,4.0


# CITY BASED RECOMMENDATION

In [74]:
#define fuction to user to recommend by using city
def citybased(city):
    popular= hotel_df.loc[hotel_df['starrating'] >= 3]
    hotel_df['city'] = hotel_df['city'].str.lower()
    citybase = hotel_df[hotel_df['city'] == city.lower()]
    citybase = citybase.sort_values(by='starrating',ascending=False)
    citybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(citybase.empty==0):
        hname=citybase[['hotelname','roomtype','city','address','starrating','url']]
        return hname
    else:
        print("No avaliable hotle in this city \nYou may like....")
        return popular

In [75]:
#if no city mention based on high rating recoomeded to user
citybased('')

No avaliable hotle in this city 
You may like....


,hotelcode,hotelname,roomtype,city,address,roomamenities,country,url,starrating
0,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
1,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Blackout curtains: ;Coffee/...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
2,31,"The Harmonie Vienna, BW Premier Collection",Comfort Double Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
3,31,"The Harmonie Vienna, BW Premier Collection",Comfort Room,Vienna,Harmoniegasse 5-7,Air conditioning: ;Blackout curtains: ;Coffee/...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
4,31,"The Harmonie Vienna, BW Premier Collection",Double Non Smoking,Vienna,Harmoniegasse 5-7,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0
...,...,...,...,...,...,...,...,...,...
45693,8948048,Hotel Merkur - West Station Lodge,room,Interlaken,Bahnhofstrasse 35,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Switzerland,https://www.agoda.com/en-gb/hotel-merkur-west-...,3.0
45699,8965903,house Jadranovo,Four-Bedroom House,Jadranovo,8 Obala,Additional toilet: ;Air conditioning: ;Coffee/...,Croatia,https://www.agoda.com/en-gb/house-jadranovo/ho...,4.0
45703,8971223,Red Lion Inn,Double Room with Private Bathroom,Cricklade,74 High Street,Air conditioning: ;Alarm clock: ;Closet: ;Coff...,United Kingdom,https://www.agoda.com/en-gb/red-lion-inn/hotel...,4.0
45704,8971223,Red Lion Inn,Twin Room with Private Bathroom,Cricklade,74 High Street,Air conditioning: ;Alarm clock: ;Closet: ;Coff...,United Kingdom,https://www.agoda.com/en-gb/red-lion-inn/hotel...,4.0


In [76]:
#recommendation for creil
citybased('creil')

,hotelname,roomtype,city,address,starrating,url
45697,L'hostellerie de la riviere,Double Room,creil,11 Route de Vaux,2.0,https://www.agoda.com/en-gb/l-hostellerie-de-l...


In [77]:
#make new hotel dataframe for more recommedndation
new_hotel_df = pd.merge(hotel_df,hrp ,on='hotelcode')
new_hotel_df.head()

,hotelcode,hotelname,roomtype,city,address,roomamenities,country,url,starrating,onsiterate,currency,maxoccupancy,ispromo,discount,mealinclusiontype
0,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,219.52,USD,1,N,0.0,Free Breakfast
1,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,231.57,USD,1,N,0.0,Free Breakfast
2,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,237.60,USD,2,N,0.0,Free breakfast for {2}
3,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,250.66,USD,2,N,0.0,Free breakfast for {2}
4,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,263.81,USD,1,N,0.0,Free Breakfast


# Data cleaning : Part 2

In [78]:
#Any null values in data?
print("Nulls value in new hotel dtaframe\n",new_hotel_df.isna().sum())
new_hotel_df['mealinclusiontype']=new_hotel_df['mealinclusiontype'].fillna('No Complimentary')
print("\n\nAfter filling nulls value in new hotel dtaframe\n",new_hotel_df.isna().sum())

Nulls value in new hotel dtaframe
 hotelcode                 0
hotelname                 0
roomtype                  0
city                      0
address                   0
roomamenities             0
country                   0
url                       0
starrating                0
onsiterate                0
currency                  0
maxoccupancy              0
ispromo                   0
discount                  0
mealinclusiontype    593430
dtype: int64


After filling nulls value in new hotel dtaframe
 hotelcode            0
hotelname            0
roomtype             0
city                 0
address              0
roomamenities        0
country              0
url                  0
starrating           0
onsiterate           0
currency             0
maxoccupancy         0
ispromo              0
discount             0
mealinclusiontype    0
dtype: int64


**Drop duplicates values from the data.**

In [79]:
# Drop duplicates values from the data.
print("shape of new hotel df is ",new_hotel_df.shape)
print("Number of duplicates in new hotel_df are ",new_hotel_df.duplicated().sum())
new_hotel_df = new_hotel_df.drop_duplicates()
print("After shape of new hotel df is ",new_hotel_df.shape)
print("After deleting number of duplicates in new hotel_df are  ", new_hotel_df.duplicated().sum())
new_hotel_df.head()

shape of new hotel df is  (1473894, 15)
Number of duplicates in new hotel_df are  325404
After shape of new hotel df is  (1148490, 15)
After deleting number of duplicates in new hotel_df are   0


,hotelcode,hotelname,roomtype,city,address,roomamenities,country,url,starrating,onsiterate,currency,maxoccupancy,ispromo,discount,mealinclusiontype
0,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,219.52,USD,1,N,0.0,Free Breakfast
1,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,231.57,USD,1,N,0.0,Free Breakfast
2,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,237.60,USD,2,N,0.0,Free breakfast for {2}
3,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,250.66,USD,2,N,0.0,Free breakfast for {2}
4,31,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,Austria,https://www.agoda.com/en-gb/the-harmonie-vienn...,4.0,263.81,USD,1,N,0.0,Free Breakfast


# RECOMMENDATION SYSTEM FOR CITY, ROOM TYPE AND PRICE BASED

In [80]:
#define a function for recommeding 
def recom_city_maxprice_roomtype(city, maxprice,roomtype):
    maxprice=int(maxprice)
    new_hotel_df['city'] = new_hotel_df['city'].str.lower()
    citybase = new_hotel_df[new_hotel_df['city'] == city.lower()] 
    
    citybase =  citybase[(citybase['onsiterate'] <= maxprice)  &  (citybase['roomtype'] == roomtype)]
    citybase = citybase.sort_values(by='onsiterate' ,ascending=True)
    citybase.drop_duplicates(subset='hotelname',keep='first',inplace=True)
    
    popular= new_hotel_df[new_hotel_df['city'] == city.lower()]
    popular = popular.sort_values(by='starrating' ,ascending=False)
    popular.drop_duplicates(subset='hotelname',keep='first',inplace=True)
    
    if(citybase.empty==0):
        hname=citybase[['hotelname','roomtype','city','address','starrating','onsiterate','url']]
        return hname
    else:
        print("Not avaliable hotel at the moment \nBut you may like this...")
        haname=popular[['hotelname','roomtype','city','address','starrating','onsiterate','url']]
        return haname.head(10)


In [81]:
#call fuction for recommendation
recom_city_maxprice_roomtype('vienna', 62,'Superior Room with Sofa Bed')

Not avaliable hotel at the moment 
But you may like this...


,hotelname,roomtype,city,address,starrating,onsiterate,url
0,"The Harmonie Vienna, BW Premier Collection",Classic Double or Twin Room,vienna,Harmoniegasse 5-7,4.0,219.52,https://www.agoda.com/en-gb/the-harmonie-vienn...
1014171,Living Hotel Kaiser Franz Joseph by Derag,Business Double Room,vienna,Sieveringer Strasse 4,4.0,76.03,https://www.agoda.com/en-gb/living-hotel-kaise...
963821,Fleming's Selection Hotel Wien-City,Superior Room,vienna,Josefstadter Str. 10-12,4.0,263.59,https://www.agoda.com/en-gb/fleming-s-selectio...
596268,Saint SHERMIN bed breakfast & champagne,Small Double Room,vienna,Rilke-Platz 7,4.0,187.87,https://www.agoda.com/en-gb/saint-shermin-bed-...
1075289,Best Western Plus Hotel Arcadia,Double Room,vienna,Max-Winter-Platz 5,4.0,110.94,https://www.agoda.com/en-gb/best-western-plus-...
1413888,Arthotel ANA Westbahn,Single Room,vienna,Pelzgasse 1,4.0,89.32,https://www.agoda.com/en-gb/arthotel-ana-westb...
1395155,Hotel & Apartments Rothensteiner,Double Room,vienna,Neustiftgasse 66,4.0,185.49,https://www.agoda.com/en-gb/hotel-apartments-r...
595903,Austria Trend Parkhotel Schonbrunn Wien,Deluxe Double Room,vienna,Hietzinger Hauptstr. 10-14,4.0,242.50,https://www.agoda.com/en-gb/austria-trend-park...
566458,Hotel Altstadt Vienna,Classic Double Room,vienna,Kirchengasse 41,4.0,263.60,https://www.agoda.com/en-gb/hotel-altstadt-vie...
566453,Hotel Am Parkring,Suite,vienna,Parkring 12,4.0,0.00,https://www.agoda.com/en-gb/hotel-am-parkring/...


In [82]:
#recom_city_maxprice_roomtype with superior Room with Sofa Bed
recom_city_maxprice_roomtype('vienna', 65,'Superior Room with Sofa Bed')

,hotelname,roomtype,city,address,starrating,onsiterate,url
1413931,Arthotel ANA Westbahn,Superior Room with Sofa Bed,vienna,Pelzgasse 1,4.0,62.48,https://www.agoda.com/en-gb/arthotel-ana-westb...


# RECOMMENDATION SYSTEM USING ANY SPECIAL FEATURES REQUIREMENT

In [83]:
#requirement based recomendation system
def requirementbased(city,price,roomtype,features):
    popular= new_hotel_df[new_hotel_df['city'] == city.lower()]
    popular = popular.sort_values(by='starrating' ,ascending=True)
    popular.drop_duplicates(subset='hotelname',keep='first',inplace=True)
    
    new_hotel_df['city']=new_hotel_df['city'].str.lower()
    new_hotel_df['roomamenities']=new_hotel_df['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=new_hotel_df[(new_hotel_df['city']==city.lower())]
    reqbased =  reqbased[(new_hotel_df['onsiterate'] <= price)  &  (reqbased['roomtype'] == roomtype)]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    
    l1 =[];l2 =[];cos=[];
    
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
        
    if(reqbased.empty==0):
      reqbased['similarity']=cos
      reqbased=reqbased.sort_values(by='similarity',ascending=False)
      reqbased.drop_duplicates(subset='hotelname',keep='first',inplace=True)
      return reqbased[['hotelname','city','roomtype','starrating','address','roomamenities','similarity']].head(10)

    else:
      print("Not avaliable hotel at the moment \nBut you may like this...")
      return popular[['hotelname','city','roomtype','starrating','address','roomamenities']].head(5)

In [84]:
#call fuction for recommendation
requirementbased('vienna',189,'Classic Double or Twin Room', 'I need Alarm clock , hair dryer and smoke detector for safety emergencies')

,hotelname,city,roomtype,starrating,address,roomamenities,similarity
0,"The Harmonie Vienna, BW Premier Collection",vienna,Classic Double or Twin Room,4.0,Harmoniegasse 5-7,air conditioning: ;alarm clock: ;bathrobes: ;c...,6
2,Best Western Plus Hotel Arcadia,vienna,Classic Double or Twin Room,4.0,Max-Winter-Platz 5,air conditioning: ;alarm clock: ;bathrobes: ;c...,6


In [85]:
#call fuction for recommendation
requirementbased('vienna',189,'Classic Superior Double or Twin Room', 'I need Alarm clock , hair dryer and smoke detector for safety emergencies')

Not avaliable hotel at the moment 
But you may like this...


,hotelname,city,roomtype,starrating,address,roomamenities
1108243,Ibis budget Wien Messe,vienna,Standard Room with 1 Double and 1 Single Bed,2.0,Lassallestrasse 7,air conditioning: ;desk: ;free wi-fi in all ro...
479696,Hotel Lucia,vienna,Double Single Use Business,3.0,Hutteldorfer Strasse 79.,air conditioning: ;free wi-fi in all rooms!: ;...
596461,Stanys - Das Apartmenthotel,vienna,Twin Capacity 1,3.0,Mariahilfer Strasse 161,air conditioning: ;free wi-fi in all rooms!: ;...
475505,Art Hotel Vienna,vienna,Single Room,3.0,Brandmayergasse 7-9,air conditioning: ;blackout curtains: ;carpeti...
407873,Hotel Haydn,vienna,Triple Room,3.0,Mariahilfer Strasse 57-59,air conditioning: ;closet: ;clothes rack: ;dai...


# CONCULSION
After all these a reccommender system from scratch is made which considers the following features while recommending

Closness to location, Onsite Rate, Room Type

Any special requests the customer needs(Using NLTK search engines)